#### 1. Import pandas library

In [52]:
import pandas as pd

#### 2. Import users table:

In [69]:
users = pd.read_csv("users_table.csv")

#### 3. Rename Id column to userId

In [70]:
users = users.rename(columns={"Id": "userId"})

#### 4. Import posts table:

In [71]:
posts = pd.read_csv("posts_table.csv")

#### 5. Rename Id column to postId and OwnerUserId to userId

In [72]:
posts = posts.rename(columns={"Id": "postId", "OwnerUserId": "userId", "CreaionDate": "CreationDate"})


#### 6. Define new dataframes for users and posts with the following selected columns:
    **users columns**: userId, Reputation,Views,UpVotes,DownVotes
    **posts columns**: postId, Score,userId,ViewCount,CommentCount

In [87]:
wanted_users_col = ['userId', 'Reputation', 'Views', 'UpVotes', 'DownVotes']
posts_columns_col = ['postId', 'Score', 'userId', 'ViewCount' ,'CommentCount']

users = users[wanted_users_col]
posts = posts[posts_columns_col]

#### 7. Merge both dataframes, users and posts. 
You will need to make a [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) of posts and users dataframes.

In [88]:
posts = posts.drop_duplicates()
users_posts = posts.merge(users, how = 'right', left_on = 'userId', right_on = 'userId', suffixes = ('_l','_r'))

#### 8. How many missing values do you have in your merged dataframe? On which columns?

In [89]:
users_posts.isnull().sum().sum()
print("Number of nulls: " + str(users_posts.isnull().sum().sum()))

columns_nonulls = users_posts.isnull().sum(axis=0) > 0
print(columns_nonulls[ columns_nonulls == True].index)

Number of nulls: 152320
Index(['postId', 'Score', 'ViewCount', 'CommentCount'], dtype='object')


#### 9. You will need to make something with missing values.  Will you clean or filling them? Explain. 
**Remember** to check the results of your code before passing to the next step

In [91]:
users_posts = users_posts.dropna(axis = 1)

#I dropped the missing values because i didn't had any criteria in order to fill the missing values and they weren't that important for the data frame to be clear

,userId,Reputation,Views,UpVotes,DownVotes
0,8.0,6764,1089,604,25
1,8.0,6764,1089,604,25
2,8.0,6764,1089,604,25
3,8.0,6764,1089,604,25
4,8.0,6764,1089,604,25
...,...,...,...,...,...
71144,55743.0,1,0,0,0
71145,55744.0,6,1,0,0
71146,55745.0,101,0,0,0
71147,55746.0,106,1,0,0


#### 10. Adjust the data types in order to avoid future issues. Which ones should be changed? 

In [95]:
#The ones to be changed are the floats bevause everything else is an int

users_posts["userId"] = users_posts["userId"].astype(int)